In [82]:
import operator
from googletrans import Translator
translator = Translator()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from polyglot.detect import Detector
from datetime import datetime, date
import numpy as np
import re
import polyglot
from polyglot.downloader import downloader
from polyglot.text import Text
import wikipedia
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
# A function that transforms the parallel data into vectors
def make_vectors(directory):
    table = dict()
    with open(directory, 'r', encoding='utf-8', errors='ignore') as f:
        next(f)
        vectors = []
        for i, line in enumerate(f):
            word,vect = line.rstrip().split(" ",1)
            vect = np.fromstring(vect, sep=' ')
            table[word] = vect
    return table

In [3]:
# Path of the cross lingual word embeddings (CLWEs)
en_path = 'wiki.multi.en.vec.txt'
table_eng = make_vectors(en_path)

In [77]:
# Function to wikify the entity
def selenium (url):
    chrome_options = Options() 
    chrome_options.add_argument("--headless") 
    driver = webdriver.Chrome(r'C:/Users/Jason/Downloads/chromedriver_win32/chromedriver.exe', options=chrome_options)
    driver.get(url)
    
    driver.find_element_by_link_text('English').click()
    name= driver.title
    LANGS = [
    'ÁáČčĎďÉéĚěÍíŇňÓóŘřŠšŤťÚúŮůÝýŽž',   # Czech
    'ÄäÖöÜüẞß',                         # German
    'ĄąĆćĘęŁłŃńÓóŚśŹźŻż',               # Polish
    ]
    pattern = r'[A-Z][a-z{langs}]+'.format(langs=''.join(LANGS))
    pattern = re.compile(pattern)
    match = pattern.findall(name)
    match = match[0:-1]
    try:
        if match [1] != None:
            return match[0] + " " + match[1]
    except:
        return match[0]

In [4]:
def token2multi_vectors(token):
        # Assign the multilingual vectors to the input words
        # assign a zero confidence to stopwords
        try:
            vect = table_eng[token].reshape(1,300)   
        except:
            vect = UNK
        return vect

In [5]:
# function that intercepts the vectors related to the time of the day
def when_def(when_dict, conf_value, lang):
    sim_today = []
    for key, value in when_dict.items():
        value = value.reshape(1,300)
        sim = cosine_similarity(lang, value)
        if sim > conf_value:
            sim_today.append(sim)
        return sum(sim_today)

In [6]:
def highest_similarity_for_keyword(similarity_language, vect):
    last_status = similarity_language[key]
    similarity = cosine_similarity(vect, value)
    if similarity > last_status:
        similarity_language[key] = similarity

In [29]:
def prep_city(from_phrase, parola, lang):
    prep = 0
    print(translator.translate(from_phrase, dest= lang).text) 
    city = input('YOU: ')
    city = translator.translate(city, dest= 'en').text 
    tok = city.lower().split()
    for i in range(len(tok)):
        zvect = token2multi_vectors(tok[i])
        if cosine_similarity(zvect, table_eng[parola].reshape(1,300)) > 0.8:
            prep += 1
        
    if prep > 0: 
        try:
            if tok[2] != None:
                CITY = tok[1].capitalize() + " " + tok[2].capitalize()      
        except:
            CITY = tok[1].capitalize()    
    else:
        CITY = city.capitalize()
    return CITY

In [75]:
# WeatherBot answer

def answer_weather(today, tomorrow, phrase_locality, phrase_when,  TODAY, TOMORROW, CITY, sorry_answer, lang):
    BOT = 'BOT:'
    if CITTA == False:
        CITY = prep_city(phrase_locality, 'in', lang)         
    else:
        try:
            CITY = CITY.split()
            if CITY[1] != None:
                CITY = CITY[0].capitalize() + " " + CITY[1].capitalize()
        except:
            CITY = "".join(CITY).capitalize()
    
    if today == False and tomorrow == False:
        ty = 0
        tw = 0
        print(BOT, translator.translate(phrase_when, dest=lang).text)
        when = input('YOU: ').lower()
        when = translator.translate(when, dest='en').text.split()
        for i in range(len(when)):
            xvect = token2multi_vectors(when[i])
            oggi = when_def(today_key, 0.8, xvect)
            domani = when_def(tomorrow_key, 0.8, xvect)
            if oggi > 0 and oggi > domani:
                ty +=1
            if domani > 0 and domani > oggi:
                tw +=1

        if ty > 0:
            today = TODAY
            try:
                url = "https://openweathermap.org/data/2.5/weather?q={}&appid=b6907d289e10d714a6e88b30761fae22".format(CITY)
                res = requests.get(url)
                data = res.json()
                temp = data['main']['temp']
                sky = data['weather'][0]['description']
                if lang != 'en':
                    print(BOT, translator.translate("There is {} and there are {} degrees in {} {}".format(sky,
                    temp,  CITY, today), dest=lang).text)
                else:
                    print(BOT, "There is {} and there are {} degrees in {} {}".format(sky, temp,CITY, today))
            except:
                print(BOT,sorry_answer)

        elif tw > 0:
            tomorrow = TOMORROW
            try:
                url = "https://openweathermap.org/data/2.5/weather?q={}&appid=b6907d289e10d714a6e88b30761fae22".format(CITY)
                res = requests.get(url)
                data = res.json()
                temp = data['main']['temp']
                sky = data['weather'][0]['description']
                if lang != 'en':
                    print(BOT, translator.translate("There is {} and there are {} degrees in {} {}".format(sky,
                    temp,  city, tomorrow), dest=lang).text)
                else:
                    print(BOT, "There is {} and there are {} degrees in {} {}".format(sky, temp,  CITY, tomorrow))
            except:
                print(BOT, translator.translate(sorry_answer, dest=lang).text)

        else:
            try:
                url = "https://openweathermap.org/data/2.5/weather?q={}&appid=b6907d289e10d714a6e88b30761fae22".format(CITY)
                res = requests.get(url)
                data = res.json()
                temp = data['main']['temp']
                sky = data['weather'][0]['description']
                if lang != 'en':
                    print(BOT, translator.translate("In this day there is {} and there are {} degrees in {}".format(sky, temp, 
                    CITY), dest=lang).text)
                else:
                    print(BOT, "In this day there is {} and there are {} degrees in {}".format(sky, temp,  CITY))
            except:
                print(BOT, translator.translate(sorry_answer, dest=lang).text)

    elif today == True and tomorrow == False:
        today = TODAY
        #try:
        url = "https://openweathermap.org/data/2.5/weather?q={}&appid=b6907d289e10d714a6e88b30761fae22".format(CITY)
        res = requests.get(url)
        data = res.json()
        temp = data['main']['temp']
        sky = data['weather'][0]['description']
        if lang != 'en':
                print(BOT, translator.translate("There is {} and there are {} degrees in {} {}".format(sky,
                temp,  CITY, today), dest=lang).text)
        else:
            print(BOT, "There is {} and there are {} degrees in {} {}".format(sky, temp,  CITY, today))
        #except:
         #   print(BOT, sorry_answer)

    elif tomorrow == True:
        tomorrow = TOMORROW
        
        try:
            url = "https://openweathermap.org/data/2.5/weather?q={}&appid=b6907d289e10d714a6e88b30761fae22".format(CITY)
            res = requests.get(url)
            data = res.json()
            temp = data['main']['temp']
            sky = data['weather'][0]['description']
            if lang != 'en':
                    print(BOT, translator.translate("There is {} and there are {} degrees in {} {}".format(sky,
                    temp,  CITY, tomorrow), dest=lang).text)
            else:
                print(BOT, "There is {} and there are {} degrees in {} {}".format(sky, temp,  CITY, tomorrow))
        except:
            print(BOT, translator.translate(sorry_answer, dest=lang).text)

In [34]:
# Wiki answers

def answer_wiki(frase_sorry, frase_noinfo, lang):
    
    if WHAT == True:
        try:
            res = ".".join(wikipedia.summary(wikipedia.search(topic, results=1)).split('.')[0:2])
            print('BOT:', res)
        except:
            print(translator.translate(frase_sorry, dest=lang).text)
            response = input('YOU: ').lower()
            response = translator.translate(response, dest='en').text
            try:
                res = ".".join(wikipedia.summary(wikipedia.search(response, results=1)).split('.')[0:2])
                print("BOT:", res)
            except:
                print(translator.translate(frase_noinfo, dest=lang), response)
    else:   
        try:
            if person == True:
                res = ".".join(wikipedia.summary(wikipedia.search(PERSON, results=1)).split('.')[0:2])
                print('BOT:', res)
            elif CITTA == True:
                res = ".".join(wikipedia.summary(wikipedia.search(CITY, results=1)).split('.')[0:2])
                print('BOT:', res)
            else:
                print(translator.translate(frase_sorry, dest=lang).text)
                response = input('YOU: ').lower()
                response = translator.translate(response, dest='en').text
                try:
                    res = ".".join(wikipedia.summary(wikipedia.search(response, results=1)).split('.')[0:2])
                    print("BOT:", res)
                except:
                    print(translator.translate(frase_noinfo, dest=lang), response)
        except:
            print(translator.translate(frase_sorry, dest=lang).text)
            response = input('YOU: ').lower()
            response = translator.translate(response, dest='en').text
            try:
                res = ".".join(wikipedia.summary(wikipedia.search(response, results=1)).split('.')[0:2])
                print("BOT:", res)
            except:
                print(translator.translate(frase_noinfo, dest=lang), response)


In [50]:
# Travelbot answers
def answer_travel(CITY, CITY2, leaving, arriving, today, tomorrow, lang, destination_phrase, from_city_phrase, sorry_phrase, when_phrase, oggi, domani):

    if CITY != None:
        CITY = CITY.split()
        try:
            if CITY[1] != None:
                CITY = CITY[0].capitalize() + " " + CITY[1].capitalize()
        except:
            CITY = "".join(CITY).capitalize()
    if CITY2 != None:
        CITY2 = CITY2.split()
        try:
            if CITY2[1] != None:
                CITY2 = CITY2[0].capitalize() + " " + CITY2[1].capitalize()
        except:
            CITY2 = "".join(CITY2).capitalize()

    if CITY2 !=None:
        if leaving == arriving:
            print(translator.translate(destination_phrase, dest=lang).text)
            tok = input('YOU: ').lower().capitalize()
            tok = translator.translate(tok, dest='en').text.split()

            try:
                if tok[1] != None:
                    tok = tok[0].capitalize() + " " + tok[1].capitalize()  
            except:
                tok = tok[0].capitalize()  
            if tok[:3] == CITY[:3]:
                arriving = CITY
                leaving = CITY2
            else:
                arriving = CITY2
                leaving = CITY
        
        if leaving != None:
            leaving = leaving
        else:
            where2start = prep_city(from_city_phrase, 'from', lang)
            # we just confront the first 3 letters of the 2 cities (in case of languages wt different suffixies)
            if where2start[:3] == CITY2[:3]:
                arriving = CITY
                leaving = CITY2
            elif where2start[:3] == CITY[:3]:
                arriving = CITY2
                leaving = CITY
            else:
                print(translator.translate(sorry_phrase, dest=lang).text)
                today = True

        if leaving != None and arriving == None:
            if leaving == CITY:
                arriving = CITY2
            elif leaving ==CITY2:
                arriving = CITY
            else:
                print(destination_phrase)
                arriving = input('YOU: ').lower().capitalize().split()
                try:
                    if arriving[1] != None:
                        arriving = arriving[0].capitalize() + " " + arriving[1].capitalize()  
                except:
                    arriving = arriving[0].capitalize() 

        if arriving != None and leaving == None:
            if arriving == CITY:
                leaving = CITY2
            elif arriving == CITY:
                leaving = CITY
            else:
                leaving = prep_city(from_city_phrase, 'from', lang)

    if CITY != None and CITY2 == None:
        if arriving != None:
            arriving = CITY
            leaving = prep_city(from_city_phrase, 'from', lang)
        elif leaving != None:
            leaving = CITY
            print(destination_phrase)
            arriving = input('YOU: ').lower().capitalize().split()
            try:
                if arriving[1] != None:
                    arriving = arriving[0].capitalize() + " " + arriving[1].capitalize()  
            except:
                arriving = arriving[0].capitalize()
        else:
            where2start = prep_city(from_city_phrase, 'from', lang)
            # we just confront the first 3 letters of the 2 cities (in case of languages wt different suffixies)
            if where2start[:3] == CITY[:3]:
                leaving = CITY
                arriving = input('YOU: ').lower().capitalize().split()
                try:
                    if arriving[1] != None:
                        arriving = arriving[0].capitalize() + " " + arriving[1].capitalize()  
                except:
                    arriving = arriving[0].capitalize()
                    
            else:
                arriving = CITY
                leaving = where2start
            

    elif CITY == None:
        leaving = prep_city(from_city_phrase, 'from', lang)
        
        arriving = input('YOU: ').lower().capitalize().split()
        try:
            if arriving[1] != None:
                arriving = arriving[0].capitalize() + " " + arriving[1].capitalize()  
        except:
            arriving = arriving[0].capitalize()

    if today == False and tomorrow == False:
        ty = 0
        tw = 0
        print(translator.translate(when_phrase, dest=lang).text)
        day = input('YOU: ').lower()
        day = translator.translate(day, dest='en').text
        for i in range(len(day)):
            vect = token2multi_vectors(day[i])
            oggi = when_def(today_key, 0.8, vect)
            domani = when_def(tomorrow_key, 0.8, vect)
            if oggi > 0 and oggi > domani:
                ty +=1
            if domani > 0 and domani > oggi:
                tw +=1
        if ty > 0:
            today = oggi
        elif tw > 0:
            tomorrow = domani
    else:
        if today == True:
            day = oggi
        elif tomorrow == True:
            day = domani
    
    if lang != 'en':
        if lang == 'ru':
            print("Я нахожу лучший путь от {} до {} {}".format(leaving, arriving, day))
        else:
            print("BOT:", translator.translate("I am Finding the best way to go from {} to {} {}".format(leaving, arriving, day), dest=lang).text)
        
    else:
        print("BOT: Finding the best way to go from {} to {} {}".format(leaving, arriving, day))

In [10]:
# Function that captures the highest similarity within the token and the keywords for a language
def highest_similarity_for_keyword(similarity_language, vect):
    last_status = similarity_language[key]
    similarity = cosine_similarity(vect, value)
    if similarity > last_status:
        similarity_language[key] = similarity

In [43]:
# A function that looks for bigram with high cosine similarity for the WeatherBot
def travel_bigram (FROM, which_city, conf):
    if which_city != None:
        if cosine_similarity(table_eng[str(FROM)].reshape(1,300), vect) > conf and next_token.capitalize() == which_city:
            leaving = which_city
            return leaving

In [96]:
# THE CHATBOT

##### KEYWORDS #############################################################################################################
# keywords for weather_bot
keywords_weather = {'snow':table_eng['snow'], 'rain':table_eng['rain'], 'weather':table_eng['weather'], 
                    'sun':table_eng['sun'], 'hailstorm':table_eng['hailstorm'], 'fog':table_eng['fog'], 
                   'temperature':table_eng['temperature'], 'degree':table_eng['degree'], 'cold': table_eng['cold'],
                   'hot':table_eng['hot'], 'warm':table_eng['warm']}

# keywords for Wikipedia
keywords_wikipedia = {'wikipedia':table_eng['wikipedia'], 'meaning': table_eng['meaning'], 'mean': table_eng['mean'], 
                     'sense': table_eng['sense']}

# keywords for travel_bot
keywords_travel = {'travel': table_eng['travel'], 'trip':table_eng['trip'], 'ride':table_eng['ride'], 'fly':table_eng['fly'],
                  'go':table_eng['go'], 'drive':table_eng['drive'], 'bus':table_eng['bus'], 'plane': table_eng['plane'],
                  'taxi':table_eng['taxi'], 'ferry': table_eng['ferry'], 'car':table_eng['car'], 'train': table_eng['train']}

# keywords for time_bot
keywords_time = {'time': table_eng['time'], 'hours': table_eng['hours'] }

today_key = {'today':table_eng['today'],  'now':table_eng['now']}
yesterday_key = {'yesterday':table_eng['yesterday']}
tomorrow_key = {'tomorrow':table_eng['tomorrow']}

#############################################################################################################################

# Vectors not found in MUSE
UNK = np.zeros((1, 300))

while True:
    
    #Entities (cities, time, people)
    CITTA = False
    CITTA2 = False
    today = False
    tomorrow = False
    person = False
    yesterday = False
    WHAT = False
    partenza = None
    arrivo = None
    leaving = None
    arriving = None

##### EMPTY DICTIONARIES ####################################################################################################

    # Empty dictionaries for the WEATHER BOT
    confidence_weather = []
    similarities_weather_en = {'snow':np.asarray([[0]]), 'rain':np.asarray([[0]]), 'weather':np.asarray([[0]]), 
                    'sun':np.asarray([[0]]), 'hailstorm':np.asarray([[0]]), 'fog':np.asarray([[0]]), 
                   'temperature':np.asarray([[0]]), 'degree':np.asarray([[0]]), 'cold':np.asarray([[0]]),
                   'hot':np.asarray([[0]]), 'warm':np.asarray([[0]])}
    confidence_wiki = []
    similarities_wiki_en = {'wikipedia':np.asarray([[0]]), 'meaning':np.asarray([[0]]), 'mean':np.asarray([[0]]), 
                    'sense':np.asarray([[0]])}
    confidence_travel = []
    similarities_travel_en = {'travel': np.asarray([[0]]), 'trip':np.asarray([[0]]), 'ride':np.asarray([[0]]), 
                'fly':np.asarray([[0]]),'go':np.asarray([[0]]), 'drive':np.asarray([[0]]), 'bus':np.asarray([[0]]),
                'plane': np.asarray([[0]]),'taxi':np.asarray([[0]]), 'ferry': np.asarray([[0]]), 'car':np.asarray([[0]]), 
                'train': np.asarray([[0]])}
    confidence_time = []
    similarities_time_en = {'hours':np.asarray([[0]]), 'time':np.asarray([[0]])}
    
######## START THE CONVERSATION #############################################################################################
    
    human = input('YOU: ')
    # break
    if human == 'bye':
        break
    
######### NER ##############################################################################################################
    LOC = []
    PER = []
    try:
        text = Text(human)
        for sent in text.sentences:
            for e in sent.entities:
                if e.tag == 'I-LOC':
                    LOC.append(e)
                elif e.tag == 'I-PER':
                    PER.append(e)
    except:
        pass
    # Capture if a city is given in the input
    CITY = None
    try: 
        if LOC[0][0] != None:
            CITY = LOC[0][0]
        if LOC[0][1] != None:
            CITY = LOC[0][0] + " "+ LOC[0][1]
    except:
        pass
    if CITY != None:
        CITY = CITY.capitalize()
        CITTA = True
    
    CITY2 = None
    try: 
        if LOC[1][0] != None:
            CITY2 = LOC[1][0]
        if LOC[1][1] != None:
            CITY2 = LOC[1][0] + " "+ LOC[1][1]
    except:
        pass
    if CITY2 != None:
        CITY2 = CITY2.capitalize()
        CITTA2 = True
    
    #Capture if a person is given in the input
    PERSON = None
    try: 
        if PER[0][0] != None:
            PERSON = PER[0][0]
        if PER[0][1] != None:
            PERSON = PER[0][0] + " "+ PER[0][1]
    except:
        pass
    if PERSON != None:
        person = True
    
    if CITY != None:
        try:
            website = website = 'https://'+str(lang)+'.wikipedia.org/wiki/'+CITY
            CITY_travel = selenium(website)
        except:
            CITY_travel = translator.translate(CITY, dest='en').text.capitalize()

    if CITY2 != None: 
        try:
            website = website = 'https://'+str(lang)+'.wikipedia.org/wiki/'+CITY2
            CITY2_travel= selenium(website)

        except:
            CITY2_travel = translator.translate(CITY2, dest='en').text.capitalize()

 #############################################################################################################################
    # Language Detector
    identif = translator.detect(human)
    lang = identif.lang
    
    human = translator.translate(human)
    
    human = human.text
    # Tokenize 
    human = human.split()
    hum = []
    
    for tok in human:
        hum.append(re.sub(r'[^\w]', '', tok))
    hum.append('EOS')
    

    for i in range (len(hum)-1):
        # normalize the input
        token = hum[i].lower()
        next_token = hum[i+1].lower()
       
        # Assign the multilingual vectors to the input words
        # assign a zero confidence to stopwords
        vect = token2multi_vectors(token)
        next_vect = token2multi_vectors(next_token)
  
         # capture if the time is provided in the input (today, tomorrow)
        oggi = when_def(today_key, 0.8, vect)
        domani = when_def(tomorrow_key, 0.8, vect)
        ieri = when_def(yesterday_key, 0.8, vect)
        if oggi > 0 and oggi > domani:
            today = True
        if domani > 0 and domani > oggi:
            tomorrow = True  
        
######## WEATHER BOT ##########################################################################################################

        # We compute the similarities between the multilingual vectors
        for key, value in keywords_weather.items():
            value = value.reshape(1,300)
            
            highest_similarity_for_keyword(similarities_weather_en, vect)
            sorted_simil_weather = sorted(similarities_weather_en.items(), key=operator.itemgetter(1), reverse=True)
        
        if sorted_simil_weather[0][1][0] > 0.8:
            confidence_weather.append(sorted_simil_weather[0][1] +2.6)
   
 ### WIKI BOT ################################################################################################################

        for key, value in keywords_wikipedia.items():
            value = value.reshape(1,300)

            highest_similarity_for_keyword(similarities_wiki_en, vect)
            sorted_simil_wiki = sorted(similarities_wiki_en.items(), key=operator.itemgetter(1), reverse=True)

            # check if the highest key-value pair is over the threshold and append for the confidence
            # CONFIDENCE IS GIVEN BY THE SUM OF CAPTURED KEYWORDS ABOVE THE THRESHOLD
            if sorted_simil_wiki[0][1][0] > 0.8:
                confidence_wiki.append(sorted_simil_wiki[0][1])

            if cosine_similarity(table_eng['who'].reshape(1,300), vect) + cosine_similarity(table_eng['is'].reshape(1,300), next_vect) > 1.9:
                    confidence_wiki.append(1)


            if cosine_similarity(table_eng['what'].reshape(1,300), vect) + cosine_similarity(table_eng['is'].reshape(1,300), next_vect) > 1.9:
                    confidence_wiki.append(2.6)
                    topic = hum[i+2:-1]
                    topic= " ".join(topic)
                    WHAT = True


            if cosine_similarity(table_eng['where'].reshape(1,300), vect) + cosine_similarity(table_eng['is'].reshape(1,300), next_vect) > 1.9:
                    confidence_wiki.append(1)    
                    
    
    
 ##### TRAVEL BOT ############################################################################################################

        for key, value in keywords_travel.items():
            value = value.reshape(1,300)
            
            highest_similarity_for_keyword(similarities_travel_en, vect)
            sorted_simil_travel = sorted(similarities_travel_en.items(), key=operator.itemgetter(1), reverse=True)

        
        if sorted_simil_travel[0][1][0] > 0.8:
            confidence_travel.append(sorted_simil_travel[0][1] +2.6)
        
        
        
        if CITY2_travel != None:
            if cosine_similarity(table_eng['from'].reshape(1,300), vect) > 0.8 and next_token.capitalize() == CITY_travel:
                leaving = CITY_travel
    
            elif cosine_similarity(table_eng['from'].reshape(1,300), vect) > 0.8 and next_token.capitalize() == CITY2_travel:
                leaving = CITY2_travel
        
            if cosine_similarity(table_eng['to'].reshape(1,300), vect) > 0.8 and next_token.capitalize() == CITY2_travel:
                arriving = CITY2_travel
                
            elif cosine_similarity(table_eng['to'].reshape(1,300), vect) > 0.8 and next_token.capitalize() == CITY_travel:
                arriving = CITY_travel
                
        if CITY_travel != None and CITY2_travel == None:
            if cosine_similarity(table_eng['from'].reshape(1,300), vect) > 0.8 and next_token.capitalize() == CITY_travel:
                leaving = CITY_travel
            elif cosine_similarity(table_eng['from'].reshape(1,300), vect) > 0.8 and next_token.capitalize() == CITY2_travel:
                leaving = CITY2_travel
            
            if cosine_similarity(table_eng['to'].reshape(1,300), vect) > 0.8 and next_token.capitalize() == CITY_travel:
                arriving = CITY_travel
            elif cosine_similarity(table_eng['to'].reshape(1,300), vect) > 0.8 and next_token.capitalize() == CITY2_travel:
                arriving = CITY2_travel
            
        if cosine_similarity(table_eng['going'].reshape(1, 300), vect) == 1:
            confidence_travel.append(-13)
                    
######## TIME BOT ############################################################################################################
        for key, value in keywords_time.items():
            value = value.reshape(1,300)
            
            highest_similarity_for_keyword(similarities_time_en, vect)
            sorted_simil_time = sorted(similarities_time_en.items(), key=operator.itemgetter(1), reverse=True)
       
        if sorted_simil_time[0][1][0] > 0.8:
            confidence_time.append(sorted_simil_time[0][1] +2.6)
        
        if cosine_similarity(table_eng['what'].reshape(1,300), vect) + cosine_similarity(table_eng['time'].reshape(1,300), next_vect) > 1.9:
                confidence_time.append(9)    
   
        sim1 = cosine_similarity(table_eng['what'].reshape(1,300), vect)
        sim2 = cosine_similarity(table_eng['hours'].reshape(1,300), next_vect)
        if  sim1 +  sim2 > 0.75:
            confidence_time.append(5)
        
 
 ####### CONFIDENCE OVERALL ###############################################################################################################

    # Compute the confidences and we select the bot with the highest confidence
    conf_weather = sum(confidence_weather)
    conf_wiki = sum(confidence_wiki)
    conf_travel= sum(confidence_travel)
    conf_time = sum(confidence_time)
    if person == True:
        conf_wiki = conf_wiki + 3
    if today or tomorrow or yesterday == True:
        conf_wiki = conf_wiki - 4
    if CITY2 == True:
        conf_travel += 3
    confidence_overall = sorted([conf_weather, conf_travel, conf_wiki, conf_time], reverse=True)
    
######## WEATHER RESPONSE #################################################################################################### 
    if confidence_overall[0] == conf_weather and confidence_overall[0] > 0:
        temp = answer_weather(today, tomorrow, "BOT: In which location?", "When?", "today", "tomorrow",CITY,
                           "I am sorry, we don't have information about this location", lang)

 ## WIKI ANSWER ###########################################################################################################       
    elif confidence_overall[0] == conf_wiki and confidence_overall[0] > 0:
        wikipedia.set_lang(lang)
        answer_wiki("BOT: Sorry I am not sure I understood, which topic were you looking for?","BOT: Sorry we don't have information about", lang)  
        
### TIME ANSWER ##########################################################################################################
    elif confidence_overall[0] == conf_time and confidence_overall[0] > 0:
        if lang == 'en':
            print("BOT: It's", datetime.now().strftime("%H:%M"))
        elif lang == 'es':
            print ("BOT: Son las", datetime.now().strftime("%H:%M"))
            
        elif lang == 'ru':
            print("BOT: Это", datetime.now().strftime("%H:%M"), "вечера")
           
        elif lang == 'et':
            print("BOT: See on", datetime.now().strftime("%H:%M"))
           
        elif lang == 'it':
            print("BOT: Sono le", datetime.now().strftime("%H:%M"))
            
#### TRAVEL ANSWER ########################################################################################################
    elif confidence_overall[0] == conf_travel and confidence_overall[0] > 0:
        
       
        
        if lang == 'en':
            answer_travel(CITY, CITY2, leaving, arriving, today, tomorrow, 'en', "BOT: Which is your destination?", 
            "BOT: From which city would you like to leave?", "BOT: I am sorry, the city you inserted doesn't coincide with the one you told me earlier",
            "BOT: When?", "today", "tomorrow")
        elif lang == 'es':
            answer_travel(CITY, CITY2, leaving, arriving, today, tomorrow, 'es', "BOT: ¿Cuál es tu destino?",
            "BOT: ¿De qué ciudad te gustaría irte?", "BOT: Lo siento, la ciudad insertada no coincide con la que me dijiste antes",
             "BOT: Cuando?", "hoy", "mañana")
        elif lang == 'it':
            answer_travel(CITY, CITY2, leaving, arriving, today, tomorrow, 'it', "BOT: qual è la tua destinazione?", 
             "BOT: Da quale città ti piacerebbe partire?", "BOT: Mi dispiace, la città che hai digitato non corrisponde a quella inserita in precedenza",
            "BOT: Quando?", "oggi", "domani")
        elif lang== 'et':
            answer_travel(CITY, CITY2, leaving, arriving, today, tomorrow, 'et', "BOT: mis on teie sihtkoht?",
            "BOT: milline linn sooviksite lahkuda?", "BOT: Mul on kahju, et sisestatud linn ei lange kokku sellega, mida sa varem ütlesid",
             "BOT: Millal?", "täna", "homme")
        elif lang == 'ru':
            answer_travel(CITY, CITY2, leaving, arriving, today, tomorrow, 'ru', "BOT: Куда ты направляешься",
            "BОТ: из какого города вы хотели бы уехать?", "БОТ: извините, введенный вами город не совпадает с тем, который вы мне сказали ранее",
            "BОТ: Когда?", "Сегодня", "Завтра")      

        
    # in case the threshold is not reached
    else:
        print("Sorry I didn't understand, could you repeat in other words?")

YOU: voglio andare a Venezia da Padova
BOT: Quando?
YOU: adesso
BOT: Sto trovando il modo migliore per andare da Padova a Venezia ora
YOU: c'e' il sole a tartu oggi?
BOT: C'è un cielo terso e oggi ci sono 19,24 gradi a Tartu
YOU: c'e' freddo a Berlino?
BOT: Quando?
YOU: adesso
BOT: In questo giorno ci sono i temporali e ci sono 21,21 gradi a Berlino
YOU: bye
